<a href="https://colab.research.google.com/github/shahdhesham/Colab-Thesis/blob/main/Final_Magicoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [3]:
# Iterative following prompt template - NOT BEST RESULTS

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Hardcoded C++ Code (modify this as needed)
cpp_code = """
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter two numbers: ";
    cin >> a >> b;
    cout << "Sum: " << a + b << endl;
    return 0;
}
"""

# 2. Load Model (do this once)
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Core Translation Function with Structured Prompts
def translate_cpp_to_c(refinement_prompt=""):
    prompt = f"""### Instruction:
Translate the following C++ code to C. Follow these rules:
1. Output ONLY raw C code (no Natural Language Text)
2. Maintain exact functionality
3. DO NOT include markdown (no ```c or ```)
4. MUST include all required headers
5. {refinement_prompt}

EXAMPLE OUTPUT:
#include <stdio.h>
int main() {{
    printf("Hello World");
    return 0;
}}


### C++ Code:
{cpp_code}

### Response (C Code):
"""  # Explicit response marker

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0,
        pad_token_id=tokenizer.eos_token_id  # Use EOS token as pad token
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract everything after the response marker
    translated_code = full_response.split("### Response (C Code):")[-1].strip()
    return translated_code

# 4. Interactive Refinement Loop
print("Initial C translation:")
initial_c = translate_cpp_to_c()
print(initial_c)

while True:
    refinement = input("\nRefinement prompt (e.g. 'use pointers', 'quit' to exit): ").strip()
    if refinement.lower() == 'quit':
        break

    refined_c = translate_cpp_to_c(refinement)
    print("\nRefined C code:")
    print(refined_c)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Initial C translation:
```c
#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}
```

Refinement prompt (e.g. 'use pointers', 'quit' to exit): start the answer from #include <stdio.h>

Refined C code:
```c
#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}
```

Refinement prompt (e.g. 'use pointers', 'quit' to exit): quit


In [4]:
# Deepseek Iterative model - BEST RESULTS NOT FOLLOWING PROMPT TEMPLATE

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Hardcoded C++ Code (modify this as needed)
cpp_code = """
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter two numbers: ";
    cin >> a >> b;
    cout << "Sum: " << a + b << endl;
    return 0;
}
"""

# 2. Load Model (do this once)
model_name = "ise-uiuc/Magicoder-S-CL-7B"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                           device_map="auto",
                                           torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Core Translation Function
def translate_cpp_to_c(refinement_prompt=""):
    prompt = f"""
    Translate this C++ to C. Rules:
    1. Output ONLY raw C code
    2. {refinement_prompt}

    C++:
    {cpp_code}

    C:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=500, temperature=0)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("C:")[-1].strip()

# 4. Interactive Refinement Loop
#print("Initial C translation:")
initial_c = translate_cpp_to_c()
print(initial_c)

while True:

    refinement = input("\nRefinement prompt (e.g. 'use pointers', or 'quit'): ")
    if refinement.lower() == 'quit':
        break

    refined_c = translate_cpp_to_c(refinement)
    #print("\nRefined C code:")
    print(refined_c)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#include <stdio.h>

int main() {
    int a, b;
    printf("Enter two numbers: ");
    scanf("%d %d", &a, &b);
    printf("Sum: %d\n", a + b);
    return 0;
}

Refinement prompt (e.g. 'use pointers', or 'quit'): quit
